## Exercise 7 - Final

This final exercise combines different parts from the previous exercises.

### Section 1

1 Search Digitraffic's REST interface service (in JSON format) for active trains with the endpoint address `/live-trains` from Helsinki station with the following search parameters:
* arrived_trains=10
* departed_trains=10
* include_nonstopping=False

In [4]:
import requests

# REST API endpoint and parameters
url = "https://rata.digitraffic.fi/api/v1/live-trains/station/HKI"
params = {
    "arrived_trains": 10,
    "departed_trains": 10,
    "include_nonstopping": "false"
}

# Fetch the data
response = requests.get(url, params=params)
data_json = response.json()

# Optional: preview first train
# data_json[0] if data_json else None

2 Read the data into the Pandas dataframe without editing, so that the column headings are as follows:

 `cancelled`,
 `commuterLineID`,
 `departureDate`,
 `operatorShortCode`,
 `operatorUICCode`,
 `runningCurrently`,
 `timeTableRows`,
 `timetableAcceptanceDate`,
 `timetableType`,
 `trainCategory`,
 `trainNumber`,
 `trainType`,
 `version`

In [6]:
import pandas as pd

# Load JSON data into a DataFrame without editing
df = pd.json_normalize(data_json)

# Ensure the expected columns exist (they may vary slightly depending on API response)
expected_columns = [
    'cancelled',
    'commuterLineID',
    'departureDate',
    'operatorShortCode',
    'operatorUICCode',
    'runningCurrently',
    'timeTableRows',
    'timetableAcceptanceDate',
    'timetableType',
    'trainCategory',
    'trainNumber',
    'trainType',
    'version'
]

# Keep only the expected columns that exist in the DataFrame
df = df[[col for col in expected_columns if col in df.columns]]

df.head()

,cancelled,commuterLineID,departureDate,operatorShortCode,operatorUICCode,runningCurrently,timeTableRows,timetableAcceptanceDate,timetableType,trainCategory,trainNumber,trainType,version
0,False,L,2025-10-06,vr,10,False,"[{'stationShortCode': 'KKN', 'stationUICCode':...",2025-07-03T04:56:52.000Z,REGULAR,Commuter,8548,HL,292329395357
1,False,P,2025-10-06,vr,10,False,"[{'stationShortCode': 'HKI', 'stationUICCode':...",2025-07-03T04:56:52.000Z,REGULAR,Commuter,8752,HL,292329292333
2,False,P,2025-10-06,vr,10,False,"[{'stationShortCode': 'HKI', 'stationUICCode':...",2025-07-03T04:56:52.000Z,REGULAR,Commuter,8754,HL,292329402538
3,False,P,2025-10-06,vr,10,True,"[{'stationShortCode': 'HKI', 'stationUICCode':...",2025-07-03T04:56:52.000Z,REGULAR,Commuter,8757,HL,292329465406
4,False,I,2025-10-06,vr,10,False,"[{'stationShortCode': 'HKI', 'stationUICCode':...",2025-07-03T04:56:52.000Z,REGULAR,Commuter,8952,HL,292329283104


3 Delete all other columns except `timeTableRows` and `trainNumber`.

In [8]:
# Keep only 'trainNumber' and 'timeTableRows'
df = df[['trainNumber', 'timeTableRows']]

df.head()

,trainNumber,timeTableRows
0,8548,"[{'stationShortCode': 'KKN', 'stationUICCode':..."
1,8752,"[{'stationShortCode': 'HKI', 'stationUICCode':..."
2,8754,"[{'stationShortCode': 'HKI', 'stationUICCode':..."
3,8757,"[{'stationShortCode': 'HKI', 'stationUICCode':..."
4,8952,"[{'stationShortCode': 'HKI', 'stationUICCode':..."


4 After this, extract the values ​​of the `timeTableRows` column from each train separately (the column value for each train contains several JSON objects) and add `trainNumber`, which identifies the train data.

In [10]:
# Expand the timeTableRows column into separate rows per train
df_expanded = pd.DataFrame(
    [(row['trainNumber'], ttr) for _, row in df.iterrows() for ttr in row['timeTableRows']],
    columns=['trainNumber', 'timeTableRow']
)

# Flatten the nested dictionary in 'timeTableRow' into columns
df_expanded = pd.json_normalize(df_expanded.to_dict(orient='records'))

df_expanded.head()

,trainNumber,timeTableRow.stationShortCode,timeTableRow.stationUICCode,timeTableRow.countryCode,timeTableRow.type,timeTableRow.trainStopping,timeTableRow.commercialStop,timeTableRow.cancelled,timeTableRow.scheduledTime,timeTableRow.actualTime,timeTableRow.differenceInMinutes,timeTableRow.causes,timeTableRow.stopSector,timeTableRow.commercialTrack,timeTableRow.trainReady.source,timeTableRow.trainReady.accepted,timeTableRow.trainReady.timestamp,timeTableRow.estimateSource,timeTableRow.liveEstimateTime
0,8548,KKN,63,FI,DEPARTURE,True,True,False,2025-10-06T20:37:00.000Z,2025-10-06T20:37:00.000Z,0.0,[],A2,2,KUPLA,True,2025-10-06T20:29:22.000Z,NaN,NaN
1,8548,TOL,830,FI,ARRIVAL,True,True,False,2025-10-06T20:38:30.000Z,2025-10-06T20:38:55.000Z,0.0,[],A4,2,NaN,NaN,NaN,NaN,NaN
2,8548,TOL,830,FI,DEPARTURE,True,True,False,2025-10-06T20:39:00.000Z,2025-10-06T20:39:20.000Z,0.0,[],A4,2,NaN,NaN,NaN,NaN,NaN
3,8548,HEK,1023,FI,ARRIVAL,False,NaN,False,2025-10-06T20:39:30.000Z,2025-10-06T20:39:55.000Z,0.0,[],NaN,,NaN,NaN,NaN,NaN,NaN
4,8548,HEK,1023,FI,DEPARTURE,False,NaN,False,2025-10-06T20:39:30.000Z,2025-10-06T20:39:55.000Z,0.0,[],NaN,,NaN,NaN,NaN,NaN,NaN


5 Then drop the following columns from the data: `causes`, `differenceInMinutes` and `commercialStop`.

In [12]:
# Columns to drop
columns_to_drop = ['timeTableRow.causes', 'timeTableRow.differenceInMinutes', 'timeTableRow.commercialStop']

# Drop columns only if they exist
df_expanded.drop(columns=[c for c in columns_to_drop if c in df_expanded.columns], inplace=True)

df_expanded.head()

,trainNumber,timeTableRow.stationShortCode,timeTableRow.stationUICCode,timeTableRow.countryCode,timeTableRow.type,timeTableRow.trainStopping,timeTableRow.cancelled,timeTableRow.scheduledTime,timeTableRow.actualTime,timeTableRow.stopSector,timeTableRow.commercialTrack,timeTableRow.trainReady.source,timeTableRow.trainReady.accepted,timeTableRow.trainReady.timestamp,timeTableRow.estimateSource,timeTableRow.liveEstimateTime
0,8548,KKN,63,FI,DEPARTURE,True,False,2025-10-06T20:37:00.000Z,2025-10-06T20:37:00.000Z,A2,2,KUPLA,True,2025-10-06T20:29:22.000Z,NaN,NaN
1,8548,TOL,830,FI,ARRIVAL,True,False,2025-10-06T20:38:30.000Z,2025-10-06T20:38:55.000Z,A4,2,NaN,NaN,NaN,NaN,NaN
2,8548,TOL,830,FI,DEPARTURE,True,False,2025-10-06T20:39:00.000Z,2025-10-06T20:39:20.000Z,A4,2,NaN,NaN,NaN,NaN,NaN
3,8548,HEK,1023,FI,ARRIVAL,False,False,2025-10-06T20:39:30.000Z,2025-10-06T20:39:55.000Z,NaN,,NaN,NaN,NaN,NaN,NaN
4,8548,HEK,1023,FI,DEPARTURE,False,False,2025-10-06T20:39:30.000Z,2025-10-06T20:39:55.000Z,NaN,,NaN,NaN,NaN,NaN,NaN


6 Fill in the missing values ​​of the `actualTime` column in each row with the values of the `scheduledTime` column.

In [14]:
# Fill missing actualTime with scheduledTime
if 'timeTableRow.actualTime' in df_expanded.columns and 'timeTableRow.scheduledTime' in df_expanded.columns:
    df_expanded['timeTableRow.actualTime'] = df_expanded['timeTableRow.actualTime'].fillna(
        df_expanded['timeTableRow.scheduledTime']
    )
else:
    # If actualTime does not exist, create it using scheduledTime
    if 'timeTableRow.scheduledTime' in df_expanded.columns:
        df_expanded['timeTableRow.actualTime'] = df_expanded['timeTableRow.scheduledTime']
    else:
        df_expanded['timeTableRow.actualTime'] = None

df_expanded.head()

,trainNumber,timeTableRow.stationShortCode,timeTableRow.stationUICCode,timeTableRow.countryCode,timeTableRow.type,timeTableRow.trainStopping,timeTableRow.cancelled,timeTableRow.scheduledTime,timeTableRow.actualTime,timeTableRow.stopSector,timeTableRow.commercialTrack,timeTableRow.trainReady.source,timeTableRow.trainReady.accepted,timeTableRow.trainReady.timestamp,timeTableRow.estimateSource,timeTableRow.liveEstimateTime
0,8548,KKN,63,FI,DEPARTURE,True,False,2025-10-06T20:37:00.000Z,2025-10-06T20:37:00.000Z,A2,2,KUPLA,True,2025-10-06T20:29:22.000Z,NaN,NaN
1,8548,TOL,830,FI,ARRIVAL,True,False,2025-10-06T20:38:30.000Z,2025-10-06T20:38:55.000Z,A4,2,NaN,NaN,NaN,NaN,NaN
2,8548,TOL,830,FI,DEPARTURE,True,False,2025-10-06T20:39:00.000Z,2025-10-06T20:39:20.000Z,A4,2,NaN,NaN,NaN,NaN,NaN
3,8548,HEK,1023,FI,ARRIVAL,False,False,2025-10-06T20:39:30.000Z,2025-10-06T20:39:55.000Z,NaN,,NaN,NaN,NaN,NaN,NaN
4,8548,HEK,1023,FI,DEPARTURE,False,False,2025-10-06T20:39:30.000Z,2025-10-06T20:39:55.000Z,NaN,,NaN,NaN,NaN,NaN,NaN


### Section 2

7 Extract the cell values ​​of the `table` element of the `data.html` file using the regex syntax and connect them to the dataframe created in section 1. **Note**: only regex should be used in the task (E.g. the use of Pandas' read_html() method is prohibited)!

In [17]:
import re


# Read HTML content
with open('data.html', 'r', encoding='utf-8') as f:
    html_content = f.read()

# Extract all table cell values using regex
cells = re.findall(r'<td.*?>(.*?)</td>', html_content, re.DOTALL)

# Extract all <tr> rows
rows_tr = re.findall(r'<tr.*?>(.*?)</tr>', html_content, re.DOTALL)

# Determine number of columns based on the first non-empty row
num_columns = 0
for tr in rows_tr:
    td_in_tr = re.findall(r'<td.*?>(.*?)</td>', tr, re.DOTALL)
    if td_in_tr:
        num_columns = len(td_in_tr)
        break

# Fallback if still zero
if num_columns == 0:
    raise ValueError("No table cells found in the HTML file.")

# Reshape flat list of cells into rows
rows = [cells[i:i+num_columns] for i in range(0, len(cells), num_columns)]

# Create DataFrame
df_html = pd.DataFrame(rows, columns=[f'col{i+1}' for i in range(num_columns)])

# Merge with train DataFrame
df_final = pd.concat([df_expanded.reset_index(drop=True), df_html.reset_index(drop=True)], axis=1)

df_final.head()

,trainNumber,timeTableRow.stationShortCode,timeTableRow.stationUICCode,timeTableRow.countryCode,timeTableRow.type,timeTableRow.trainStopping,timeTableRow.cancelled,timeTableRow.scheduledTime,timeTableRow.actualTime,timeTableRow.stopSector,...,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10
0,8548,KKN,63,FI,DEPARTURE,True,False,2025-10-06T20:37:00.000Z,2025-10-06T20:37:00.000Z,A2,...,\n 2019-08-06T04:48:00....,\n False\n,\n \n,\n FI\n,\n 2019-08-06T04:48:00....,\n JVS\t\n,\n 1272\n,\n False\n,\n DEPARTURE\n ...,\n 150\n
1,8548,TOL,830,FI,ARRIVAL,True,False,2025-10-06T20:38:30.000Z,2025-10-06T20:38:55.000Z,A4,...,\n 2019-08-06T06:04:39....,\n False\n,\n\n,\n FI\n,\n 2019-08-06T06:04:39....,\n R702\n,\n 98998\n,\n False\n,\n ARRIVAL\n ...,\n 150\n
2,8548,TOL,830,FI,DEPARTURE,True,False,2025-10-06T20:39:00.000Z,2025-10-06T20:39:20.000Z,A4,...,\n 2019-08-06T07:03:12....,\n False\n,\n \n,\n FI\n,\n 2019-08-06T07:03:12....,\n ILA\n,\n 98998\n,\n 9\n,\n ARRIVAL\n ...,\n 8337\n
3,8548,HEK,1023,FI,ARRIVAL,False,False,2025-10-06T20:39:30.000Z,2025-10-06T20:39:55.000Z,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,8548,HEK,1023,FI,DEPARTURE,False,False,2025-10-06T20:39:30.000Z,2025-10-06T20:39:55.000Z,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Section 3

8 Make a SQLite database and insert the dataframe obtained as a result of section 2 to the database.

In [20]:
# Convert all columns containing lists/dicts to strings
for col in df_final.columns:
    df_final[col] = df_final[col].apply(lambda x: str(x) if isinstance(x, (list, dict)) else x)

# Now insert into SQLite
import sqlite3

conn = sqlite3.connect('train_data.db')
df_final.to_sql('live_trains', conn, if_exists='replace', index=False)

# Verify insertion
pd.read_sql('SELECT * FROM live_trains LIMIT 5', conn)

,trainNumber,timeTableRow.stationShortCode,timeTableRow.stationUICCode,timeTableRow.countryCode,timeTableRow.type,timeTableRow.trainStopping,timeTableRow.cancelled,timeTableRow.scheduledTime,timeTableRow.actualTime,timeTableRow.stopSector,...,col1,col2,col3,col4,col5,col6,col7,col8,col9,col10
0,8548,KKN,63,FI,DEPARTURE,1,0,2025-10-06T20:37:00.000Z,2025-10-06T20:37:00.000Z,A2,...,\n 2019-08-06T04:48:00....,\n False\n,\n \n,\n FI\n,\n 2019-08-06T04:48:00....,\n JVS\t\n,\n 1272\n,\n False\n,\n DEPARTURE\n ...,\n 150\n
1,8548,TOL,830,FI,ARRIVAL,1,0,2025-10-06T20:38:30.000Z,2025-10-06T20:38:55.000Z,A4,...,\n 2019-08-06T06:04:39....,\n False\n,\n\n,\n FI\n,\n 2019-08-06T06:04:39....,\n R702\n,\n 98998\n,\n False\n,\n ARRIVAL\n ...,\n 150\n
2,8548,TOL,830,FI,DEPARTURE,1,0,2025-10-06T20:39:00.000Z,2025-10-06T20:39:20.000Z,A4,...,\n 2019-08-06T07:03:12....,\n False\n,\n \n,\n FI\n,\n 2019-08-06T07:03:12....,\n ILA\n,\n 98998\n,\n 9\n,\n ARRIVAL\n ...,\n 8337\n
3,8548,HEK,1023,FI,ARRIVAL,0,0,2025-10-06T20:39:30.000Z,2025-10-06T20:39:55.000Z,None,...,None,None,None,None,None,None,None,None,None,None
4,8548,HEK,1023,FI,DEPARTURE,0,0,2025-10-06T20:39:30.000Z,2025-10-06T20:39:55.000Z,None,...,None,None,None,None,None,None,None,None,None,None
